<h1>Welcome!</h1>
<p>Thanks for reviewing my project.<br><br>
Here, I will be trying to scrape the data from the given url. <br>
------------------------------------------------------------------------------<br><br>
<h3>First, I will import and install everything I will need to scrape the site and then turn it into a DataFrame.</h3>

In [162]:
!conda install BeautifulSoup4 --yes
print('installation complete')

import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

print('imports complete')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

installation complete
imports complete


<h3>Then I need to actually scrape the site and do some BeautifulSoup methods in order to get the table into a python string</h3>

In [163]:
webpage = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
print(webpage)

#Create the BeautifulSoup object
soup = BeautifulSoup(webpage.content,'html.parser')

#Get the table
soup_table = soup.find('table')

#Turn the table into a string
textonly = soup_table.get_text(',')

<Response [200]>


<h3> I then take the newly formed string and separate it into a list, then clean up the list, separate it into sublists by the column I will eventually put into my dictionary, and then create a dictionary in order to easily create a DataFrame.

In [164]:
#Turn the string into a list
full_list = textonly.split('\n')

#Clean the list
clean_list = []
for i in full_list:
    if i != ',' and i != '':
        i = i.replace(',','')
        clean_list.append(i)
columns = clean_list[0:3]
postcodes = []
for i in range(3, len(clean_list), 3):
    postcodes.append(clean_list[i])
boroughs = []
for i in range(4, len(clean_list), 3):
    boroughs.append(clean_list[i])
neighborhoods = []
for i in range(5, len(clean_list), 3):
    neighborhoods.append(clean_list[i])
data = {'Postcode':postcodes, 'Borough':boroughs, 'Neighbourhood':neighborhoods}

<h3>I then pass the dict into my new DataFrame and clean up the DataFrame as per the instructions in the project (no Boroughs that are 'Not assigned', grouping the 'Neighbourhood' column by Postcode and Borough, etc.). Obviously this took a very long time and a lot of revisions, but I only included the code that finally worked.

In [166]:

#Turn list into a dataframe
df = pd.DataFrame(data)

#Clean the DataFrame
df = df.drop(df[df.Borough=='Not assigned'].index)
df.reset_index(drop=True, inplace=True)
for i in range(0, len(df.Neighbourhood)):
    if df.Neighbourhood.iloc[i] == 'Not assigned':
        df.Neighbourhood.iloc[i] = df.Borough.iloc[i]
df_grouped = df.groupby(['Postcode', 'Borough'])
df_neighs = df.groupby(['Postcode', 'Borough']).Neighbourhood.unique()
df1 = df_neighs.to_frame()
df1.reset_index(inplace=True)
df1_neigh = df1.Neighbourhood.apply(lambda x: ', '.join(x))
df1 = df1[['Postcode', 'Borough']]
df1['Neighbourhood'] = df1_neigh

print(df1)

    Postcode      Borough                                      Neighbourhood
0        M1B  Scarborough                                     Rouge, Malvern
1        M1C  Scarborough             Highland Creek, Rouge Hill, Port Union
2        M1E  Scarborough                  Guildwood, Morningside, West Hill
3        M1G  Scarborough                                             Woburn
4        M1H  Scarborough                                          Cedarbrae
..       ...          ...                                                ...
98       M9N         York                                             Weston
99       M9P    Etobicoke                                          Westmount
100      M9R    Etobicoke  Kingsview Village, Martin Grove Gardens, Richv...
101      M9V    Etobicoke  Albion Gardens, Beaumond Heights, Humbergate, ...
102      M9W    Etobicoke                                          Northwest

[103 rows x 3 columns]


<h3>Finally, I print up the .shape method in order to show the total number of rows in my DataFrame. Hopefully it is correct!

In [167]:
print(df1.shape)

(103, 3)
